In [ ]:
#For Broiler and Hen
import os
import cv2
import torch
from groundingdino.util.inference import Model, annotate
from ultralytics import YOLO
import numpy as np
from tqdm import tqdm
import argparse
import supervision as sv

# Paths to models and images
GROUNDING_DINO_CONFIG = "groundingdino/config/GroundingDINO_SwinB_cfg.py"  #can use SwinB and SwinT
GROUNDING_DINO_CHECKPOINT = "weights/groundingdino_swinb_cogcoor.pth"

DATASET_NAME = ""

DATASET_DIR = f"datasets/{DATASET_NAME}"
IMAGE_INPUT = f"datasets/{DATASET_NAME}/images/val"
LABELS_OUTPUT_DIR = f"datasets/{DATASET_NAME}/labels/val2"
ANNOTATED_IMG_OUTPUT = f"datasets/{DATASET_NAME}/out_val"
YOLO_TRAIN_DATA_PATH = "datasets/broiler/data.yaml"
YOLO_TEST_PATH = f"datasets/{DATASET_NAME}/images/test/"


# change for apple macbook
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

# Initialize Grounding DINO
grounding_dino_model = Model(GROUNDING_DINO_CONFIG, GROUNDING_DINO_CHECKPOINT, device)

# Define the classes or queries for Grounding DINO
queries = [""]  # Example queries

# Ensure output directory exists
os.makedirs(LABELS_OUTPUT_DIR, exist_ok=True)
os.makedirs(ANNOTATED_IMG_OUTPUT, exist_ok=True)


box_annotator = sv.BoxAnnotator(color_lookup=sv.ColorLookup.INDEX)
label_annotator = sv.LabelAnnotator(color_lookup=sv.ColorLookup.INDEX)

def filter_detection_by_area (detections: sv.Detections):
    mean_h = np.mean([y2-y1 for _, y1 , _, y2 in detections.xyxy])
    mean_w = np.mean([x2-x1 for x1, _ , x2, _ in detections.xyxy])

    std_h = np.std([y2-y1 for _, y1 , _, y2 in detections.xyxy])
    std_w = np.std([x2-x1 for x1, _ , x2, _ in detections.xyxy])

    area_indices = []
    for _xyxy, _, _, _, _, _ in detections:
        x1, y1, x2, y2 = _xyxy
        val = False if (x2-x1 >= mean_w + std_w ) or (y2-y1 > mean_h + std_h)  else True
        area_indices.append(val)
    return detections[area_indices]

images_map = {}
detections_map = {}
# # Process each image
progress_bar = tqdm(os.listdir(IMAGE_INPUT), desc="Labeling images")
for image_name in progress_bar:
    progress_bar.set_description(desc=f"Labeling {image_name}", refresh=True)
    image_path = os.path.join(IMAGE_INPUT, image_name)

    image = cv2.imread(image_path)
    images_map[image_name] = image.copy()

    detections = grounding_dino_model.predict_with_classes(
        image=image,
        classes=queries,
        box_threshold=0.13,
        text_threshold=0.2,
    )
        
    detections = detections.with_nms(threshold=1)   #chance a confidence interval to remove multiple boxes.
    detections.class_id = np.zeros(len(detections.xyxy))
    detections = filter_detection_by_area(detections)
    detections_map[image_name] = detections

    labels = [f"{queries[int(class_id)]} {confidence:0.2f}" for _, _, confidence, class_id, _, _ in detections]
    annotated_frame = cv2.cvtColor(image.copy(), cv2.COLOR_RGB2BGR)
    annotated_image = box_annotator.annotate(scene=annotated_frame, detections=detections)
    annotated_image = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    cv2.imwrite(ANNOTATED_IMG_OUTPUT + "/" + image_name, annotated_image)
            
dataset = sv.DetectionDataset(
    queries, images_map, detections_map
)

dataset.as_yolo(
    annotations_directory_path=LABELS_OUTPUT_DIR,
    min_image_area_percentage=0.01,
    data_yaml_path=DATASET_DIR + "/data.yaml",
)
        


yolo_model = YOLO('best.pt')
yolo_model.to(device)
results = yolo_model.train(data='/data.yaml', epochs=10, batch=8, imgsz=640, name="", project="")
metrics = yolo_model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps  # a list contains map50-95(B) of each category

result = yolo_model.predict(source='/images/test', save=True)


In [ ]:
#For Behavior
import os
import cv2
import torch
from groundingdino.util.inference import Model
import numpy as np
from tqdm import tqdm
import supervision as sv
from ultralytics import YOLO

# Paths to models and images
GROUNDING_DINO_CONFIG = "groundingdino/config/GroundingDINO_SwinB_cfg.py"  #can use SwinB and SwinT
GROUNDING_DINO_CHECKPOINT = "weights/groundingdino_swinb_cogcoor.pth"

DATASET_NAME = ""

DATASET_DIR = f"datasets/{DATASET_NAME}"
IMAGE_INPUT = f"datasets/{DATASET_NAME}/images/val"
LABELS_OUTPUT_DIR = f"datasets/{DATASET_NAME}/labels/val2"
ANNOTATED_IMG_OUTPUT = f"datasets/{DATASET_NAME}/out_val"
YOLO_TEST_PATH = f"datasets/{DATASET_NAME}/images/test/"

# Set the device for running models (MPS for MacBook, CUDA for GPU, CPU otherwise)
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

# Initialize Grounding DINO
grounding_dino_model = Model(GROUNDING_DINO_CONFIG, GROUNDING_DINO_CHECKPOINT, device)

# Define the classes or queries for Grounding DINO (your behavior classes)
queries = [
    "", "", "", "", "", 
    "", "", "", "", "", ""
]

# Load your pre-trained YOLO model
yolo_model = YOLO('best.pt')  # Replace with the path to your best YOLO model
yolo_model.to(device)

# Ensure output directory exists
os.makedirs(LABELS_OUTPUT_DIR, exist_ok=True)
os.makedirs(ANNOTATED_IMG_OUTPUT, exist_ok=True)

# Box and label annotators
box_annotator = sv.BoxAnnotator(color_lookup=sv.ColorLookup.INDEX)
label_annotator = sv.LabelAnnotator(color_lookup=sv.ColorLookup.INDEX)

# Function to filter detections by area
def filter_detection_by_area(detections: sv.Detections):
    mean_h = np.mean([y2 - y1 for _, y1, _, y2 in detections.xyxy])
    mean_w = np.mean([x2 - x1 for x1, _, x2, _ in detections.xyxy])

    std_h = np.std([y2 - y1 for _, y1, _, y2 in detections.xyxy])
    std_w = np.std([x2 - x1 for x1, _, x2, _ in detections.xyxy])

    area_indices = []
    for _xyxy, _, _, _, _, _ in detections:
        x1, y1, x2, y2 = _xyxy
        val = False if (x2 - x1 >= mean_w + std_w) or (y2 - y1 > mean_h + std_h) else True
        area_indices.append(val)
    return detections[area_indices]

# Function to convert bounding box to YOLO format
def convert_to_yolo_format(detection, image_shape):
    x1, y1, x2, y2 = detection.xyxy
    width, height = image_shape[1], image_shape[0]

    # Normalize coordinates by image dimensions
    x_center = (x1 + x2) / 2 / width
    y_center = (y1 + y2) / 2 / height
    w = (x2 - x1) / width
    h = (y2 - y1) / height

    return [0, x_center, y_center, w, h]  # 0 corresponds to the first class (e.g., Feeding)

# Function to save YOLO formatted labels
def save_yolo_labels(image_name, detections, labels_output_dir):
    label_file = os.path.join(labels_output_dir, image_name.replace('.jpg', '.txt'))
    
    with open(label_file, 'w') as f:
        for detection in detections:
            yolo_format = convert_to_yolo_format(detection, images_map[image_name].shape)
            f.write(" ".join(map(str, yolo_format)) + "\n")

# Ensure output directory exists for labels and annotated images
os.makedirs(LABELS_OUTPUT_DIR, exist_ok=True)
os.makedirs(ANNOTATED_IMG_OUTPUT, exist_ok=True)

# Load and process images from the dataset
images_map = {}
detections_map = {}
progress_bar = tqdm(os.listdir(IMAGE_INPUT), desc="Labeling images")
for image_name in progress_bar:
    progress_bar.set_description(desc=f"Labeling {image_name}", refresh=True)
    image_path = os.path.join(IMAGE_INPUT, image_name)

    image = cv2.imread(image_path)
    images_map[image_name] = image.copy()

    # Grounding DINO inference
    detections = grounding_dino_model.predict_with_classes(
        image=image,
        classes=queries,  # Use the full set of behavior queries
        box_threshold=0.13,
        text_threshold=0.2,
    )
    detections = detections.with_nms(threshold=1)  # Confidence interval to remove multiple boxes
    detections.class_id = np.zeros(len(detections.xyxy))  # Set the class_id to 0 for behavior classes
    detections = filter_detection_by_area(detections)
    detections_map[image_name] = detections

    # Annotate with labels
    labels = [f"{queries[int(class_id)]} {confidence:0.2f}" for _, _, confidence, class_id, _, _ in detections]
    annotated_frame = cv2.cvtColor(image.copy(), cv2.COLOR_RGB2BGR)
    annotated_image = box_annotator.annotate(scene=annotated_frame, detections=detections)
    annotated_image = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    cv2.imwrite(os.path.join(ANNOTATED_IMG_OUTPUT, image_name), annotated_image)

# Process test images with YOLO model and save the labels in YOLO format
test_images = os.listdir(YOLO_TEST_PATH)
for image_name in test_images:
    image_path = os.path.join(YOLO_TEST_PATH, image_name)
    image = cv2.imread(image_path)

    # Run YOLO prediction
    results = yolo_model(image)

    # Extract detections (bounding boxes and scores)
    detections = results[0].boxes.xyxy  # Get the bounding box coordinates
    scores = results[0].boxes.conf  # Get the confidence scores
    class_ids = results[0].boxes.cls  # Get the class IDs

    # Filter detections based on a confidence threshold
    detections = [detection for detection, score in zip(detections, scores) if score > 0.5]

    # Save the YOLO formatted labels for this image
    save_yolo_labels(image_name, detections, LABELS_OUTPUT_DIR)

    # Optionally, annotate and save the image with bounding boxes
    annotated_image = image.copy()
    for detection in detections:
        x1, y1, x2, y2 = detection
        cv2.rectangle(annotated_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # Save the annotated image
    annotated_image_path = os.path.join(ANNOTATED_IMG_OUTPUT, image_name)
    cv2.imwrite(annotated_image_path, annotated_image)
